# P2 Exploration

Basic exploration will be done in this jupyter notebook.

In [ ]:
from helpers import *
import pandas as pd
 
# Just fill this with wherever is your main data folder is (copy the path just before 'MovieSummaries'):
file_source = "/Users/shrinidhivelan/Downloads/"

# Same goes for the reviews dataset : but simply copy the entire path of where your reviews data is... 
reviews_path = "/Users/shrinidhivelan/Downloads/TMDB_movie_dataset_v11.csv"

# replace the following by the direct path of where your bechdel test movie data is:
bechdel_path = "/Users/shrinidhivelan/Downloads/movies.csv"

### creating dataset by extracting 
MovieMetadata_df, CharacterMetadata_df, names_df, plot_summaries_df, tvTropes_df, merged_df = create_datasets(file_source, reviews_path)

## We merge the data on the bechdel test : 
There are more lines as some movies occur twice (redone) in the movieMetadata dataframe

In [ ]:
# The bechdel data contains ~1700 lines, to be separated into train and test sets in order to create a model to predict the binary result
#### 80% train and 20% test for instance, this will be used to merge main data into this : 
bechdel_data = pd.read_csv(bechdel_path)
bechdel_data = bechdel_data[['year', 'title', 'binary','budget']]
bechdel_data_merged = bechdel_data.merge(MovieMetadata_df, how='left', left_on = 'title', right_on = 'Movie name')

# Define the columns to check for NaN values
columns_to_check = [
    'Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 
    'Movie release date', 'Movie box office revenue', 'Movie runtime', 
    'Movie languages (Freebase ID:name tuples)', 
    'Movie countries (Freebase ID:name tuples)', 
    'Movie genres (Freebase ID:name tuples)'
]

# Remove rows where all specified columns are NaN
bechdel_data_merged = bechdel_data_merged.dropna(subset=columns_to_check, how='all')

# Display the DataFrame to confirm rows are removed - around ~300 lines are removed
display(bechdel_data_merged)

,year,title,binary,budget,Wikipedia movie ID,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages (Freebase ID:name tuples),Movie countries (Freebase ID:name tuples),Movie genres (Freebase ID:name tuples),Year
4,2013,42,FAIL,40000000,36239136.0,/m/0k2jpkj,42,2013-04-12,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01z02hx"": ""Sports"", ""/m/03bxz7"": ""Biograp...",2013
5,2013,47 Ronin,FAIL,225000000,32806120.0,/m/0h3rn9v,47 Ronin,NaN,NaN,132.0,{},"{""/m/03_3d"": ""Japan""}","{""/m/0gw5n2f"": ""Japanese Movies""}",<NA>
6,2013,47 Ronin,FAIL,225000000,29769606.0,/m/0fp_dsj,47 Ronin,2012-11-21,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama"", ""/m/02kdv5l"": ""Action""...",2012
7,2013,A Good Day to Die Hard,FAIL,92000000,34228306.0,/m/0hhgg_1,A Good Day to Die Hard,2013-02-14,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th...",2013
10,2013,After Earth,FAIL,130000000,34429259.0,/m/0hhgh70,After Earth,2013-06-07,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/06n90"": ""Science Fiction"", ""/m/02kdv5l"": ...",2013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2341,1971,Shaft,FAIL,53012938,973155.0,/m/03vpf7,Shaft,2000-06-16,107196498.0,99.0,"{""/m/02h40lc"": ""English Language"", ""/m/04306rv...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/0lsxr"": ""Crime F...",2000
2342,1971,Straw Dogs,FAIL,25000000,196176.0,/m/01bwgr,Straw Dogs,1971-11-03,11148828.0,118.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/07s9rl0"": ""Drama""}",1971
2343,1971,Straw Dogs,FAIL,25000000,29657080.0,/m/084nr2g,Straw Dogs,2011-09-16,10324441.0,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/02wtdps"": ""Crime...",2011
2344,1971,The French Connection,FAIL,2200000,99463.0,/m/0pd64,The French Connection,1971-10-07,51700000.0,104.0,"{""/m/064_8sq"": ""French Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/0lsxr"": ""Crime F...",1971


In [18]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Step 1: Load and preprocess the data
df = bechdel_data_merged  

# Specify the target and feature columns
target = 'result'
features = [
    'year', 'budget', 'Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 
    'Movie release date', 'Movie box office revenue', 'Movie runtime', 
    'Movie languages (Freebase ID:name tuples)', 'Movie countries (Freebase ID:name tuples)', 
    'Movie genres (Freebase ID:name tuples)', 'Year'
]

# Convert 'FAIL' to 0 and 'PASS' to 1 in the 'binary' column
df['result'] = df['binary'].replace({'FAIL': 0, 'PASS': 1})


# Drop rows where target is NaN
df = df.dropna(subset=[target])

# Separate features and target variable (X and y constantly)
X = df[features]
y = df[target]

# Step 2: Preprocess features
# Define which columns need specific preprocessing
numeric_features = ['year', 'budget', 'Movie box office revenue', 'Movie runtime', 'Year']
categorical_features = [
    'Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 
    'Movie languages (Freebase ID:name tuples)', 'Movie countries (Freebase ID:name tuples)', 
    'Movie genres (Freebase ID:name tuples)'
]

# Define preprocessors
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Step 3: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)#, random_state=42)

# Step 4: Define and train the model pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier())])#(random_state=42))])

# Train the model
model.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"ROC AUC Score: {roc_auc:.2f}")


/var/folders/p6/m1p5k4zj0yn9mk66d_dh7pn40000gn/T/ipykernel_24135/3015687340.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['result'] = df['binary'].replace({'FAIL': 0, 'PASS': 1})


Accuracy: 0.60
F1 Score: 0.46
ROC AUC Score: 0.59


In [22]:
plot_summaries_df
plot_summaries_df_merged = plot_summaries_df.merge(MovieMetadata_df, how='left', left_on = 'Wikipedia movie ID', right_on = 'Wikipedia movie ID')


In [24]:
plot_summaries_df_merged

# The bechdel data contains ~1700 lines, to be separated into train and test sets in order to create a model to predict the binary result
#### 80% train and 20% test for instance, this will be used to merge main data into this : 
bechdel_data = pd.read_csv(bechdel_path)
bechdel_data = bechdel_data[['year', 'title', 'binary','budget']]
bechdel_data_merged_new = bechdel_data.merge(plot_summaries_df_merged, how='left', left_on = 'title', right_on = 'Movie name')

# Define the columns to check for NaN values
columns_to_check = [
    'Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 
    'Movie release date', 'Movie box office revenue', 'Movie runtime', 
    'Movie languages (Freebase ID:name tuples)', 
    'Movie countries (Freebase ID:name tuples)', 
    'Movie genres (Freebase ID:name tuples)'
]

# Remove rows where all specified columns are NaN
bechdel_data_merged_new = bechdel_data_merged_new.dropna(subset=columns_to_check, how='all')

# Display the DataFrame to confirm rows are removed - around ~300 lines are removed
display(bechdel_data_merged_new)

,year,title,binary,budget,Wikipedia movie ID,Summaries,Freebase movie ID,Movie name,Movie release date,Movie box office revenue,Movie runtime,Movie languages (Freebase ID:name tuples),Movie countries (Freebase ID:name tuples),Movie genres (Freebase ID:name tuples),Year
5,2013,47 Ronin,FAIL,225000000,29769606.0,{{main}} The outcast Kai joins a group of Sam...,/m/0fp_dsj,47 Ronin,2012-11-21,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/07s9rl0"": ""Drama"", ""/m/02kdv5l"": ""Action""...",2012
9,2013,After Earth,FAIL,130000000,34429259.0,One thousand years after cataclysmic events fo...,/m/0hhgh70,After Earth,2013-06-07,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/06n90"": ""Science Fiction"", ""/m/02kdv5l"": ...",2013
10,2013,American Hustle,PASS,40000000,15697462.0,Katt Williams decides that he wants to make it...,/m/03nqd13,American Hustle,2007-06-01,NaN,NaN,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01z4y"": ""Comedy""}",2007
11,2013,August: Osage County,PASS,25000000,37210334.0,The movie is based upon a play by the same nam...,/m/0n52lw7,August: Osage County,2013,NaN,NaN,{},"{""/m/09c7w0"": ""United States of America""}","{""/m/05p553"": ""Comedy film"", ""/m/07s9rl0"": ""Dr...",2013
15,2013,Carrie,PASS,30000000,6584119.0,{{Plot}} Carrie Meeber ([[Jennifer Jones leav...,/m/0gcn0w,Carrie,1952-07-17,1800000.0,118.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/068d7h"": ""Romantic drama"", ""/m/02l7c8"": ""...",1952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2106,1971,Shaft,FAIL,53012938,76345.0,"{{Plot}} Shaft, a private detective, emerges f...",/m/0k47y,Shaft,1971-06-25,NaN,98.0,"{""/m/02h40lc"": ""English Language"", ""/m/0t_2"": ...","{""/m/09c7w0"": ""United States of America""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/01jfsb"": ""Th...",1971
2107,1971,Straw Dogs,FAIL,25000000,29657080.0,Los Angeles scriptwriter David Sumner and his...,/m/084nr2g,Straw Dogs,2011-09-16,10324441.0,110.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/02wtdps"": ""Crime...",2011
2108,1971,Straw Dogs,FAIL,25000000,196176.0,"David Sumner , a timid American mathematician,...",/m/01bwgr,Straw Dogs,1971-11-03,11148828.0,118.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/07s9rl0"": ""Drama""}",1971
2109,1971,The French Connection,FAIL,2200000,99463.0,The film revolves around the smuggling of narc...,/m/0pd64,The French Connection,1971-10-07,51700000.0,104.0,"{""/m/064_8sq"": ""French Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/0lsxr"": ""Crime F...",1971


In [31]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

# Step 1: Load and preprocess the data
df = bechdel_data_merged_new  

# Specify the target and feature columns
target = 'result'
features = [
    'year', 'budget', 'Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 
    'Movie release date', 'Movie box office revenue', 'Movie runtime', 
    'Movie languages (Freebase ID:name tuples)', 'Movie countries (Freebase ID:name tuples)', 
    'Movie genres (Freebase ID:name tuples)', 'Year', 'Summaries'
]

# Convert 'FAIL' to 0 and 'PASS' to 1 in the 'binary' column
df['result'] = df['binary'].replace({'FAIL': 0, 'PASS': 1})


# Drop rows where target is NaN
df = df.dropna(subset=[target])

# Separate features and target variable (X and y constantly)
X = df[features]
y = df[target]

# Step 2: Preprocess features
# Define which columns need specific preprocessing
numeric_features = ['year', 'budget', 'Movie box office revenue', 'Movie runtime', 'Year']
categorical_features = [
    'Wikipedia movie ID', 'Freebase movie ID', 'Movie name', 'Summaries'
    , 'Movie countries (Freebase ID:name tuples)', 
    'Movie genres (Freebase ID:name tuples)'
]

# Define preprocessors
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine preprocessors
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Step 3: Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)#, random_state=42)

# Step 4: Define and train the model pipeline
model2 = Pipeline(steps=[('preprocessor', preprocessor),
                        ('classifier', RandomForestClassifier())])#(random_state=42))])

# Train the model
model2.fit(X_train, y_train)

# Step 5: Evaluate the model
y_pred = model2.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.2f}")
print(f"F1 Score: {f1:.2f}")
print(f"ROC AUC Score: {roc_auc:.2f}")


/var/folders/p6/m1p5k4zj0yn9mk66d_dh7pn40000gn/T/ipykernel_24135/1619350782.py:23: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['result'] = df['binary'].replace({'FAIL': 0, 'PASS': 1})


Accuracy: 0.59
F1 Score: 0.44
ROC AUC Score: 0.57
